In [1]:
# Verify kernel is using the correct environment
import sys
print(f"Python executable: {sys.executable}")
print(f"Python version: {sys.version}")
print(f"Using UV environment: {'/root/.venv' in sys.executable}")

Python executable: /root/.venv/bin/python
Python version: 3.11.14 (main, Oct 14 2025, 21:26:53) [Clang 20.1.4 ]
Using UV environment: True


In [2]:
# Import required libraries
from __future__ import annotations

import os
import random
from collections import Counter
from pathlib import Path
import sys
from typing import List

# Force CPU to avoid GPU memory issues with large graphs
os.environ.setdefault('JAX_PLATFORMS', 'cpu')

import jax
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt

# Add tools directory to path
sys.path.insert(0, str(Path.cwd() / 'tools'))

from tv_graph import TVGraph
from thrml import Block, BlockGibbsSpec, FactorSamplingProgram, SamplingSchedule, sample_states
from thrml.block_management import block_state_to_global
from thrml.block_sampling import sample_blocks
from thrml.models import CategoricalEBMFactor, CategoricalGibbsConditional
from thrml.models.ebm import DEFAULT_NODE_SHAPE_DTYPES
from thrml.pgm import CategoricalNode

## Configuration

In [ ]:
# Configuration parameters
INPUT_FOLDER = Path("input/fcc-seed87-st50-ch30-post-auction-50st-15ch")
LAMBDA_CONFLICT = 8.0
LAMBDA_DOMAIN = 100.0
WARMUP_STEPS = 200
NUM_SAMPLES = 200
STEPS_PER_SAMPLE = 1
SEED = 0

# Blocking strategy parameters
BLOCK_SIZE_GEOGRAPHIC = 5  # Stations per geographic block
BLOCK_SIZE_RANDOM = 3      # Stations per random block

## Load Graph Data

In [ ]:
graph = TVGraph(INPUT_FOLDER)
print(f"Loaded graph with {graph.station_count:,} stations and {graph.channel_count:,} channels.")

## Helper Functions

In [ ]:
def make_domain_factor(graph: TVGraph, nodes: list[CategoricalNode], penalty: float) -> CategoricalEBMFactor:
    station_count = len(nodes)
    channel_count = graph.channel_count
    weights = np.full((station_count, channel_count), -float(penalty), dtype=np.float32)
    for station in graph.stations_by_id.values():
        if station.station_index is None:
            raise ValueError(f"Station {station.station_id} is missing a contiguous index.")
        weights[station.station_index, station.domain_indices] = 0.0
    return CategoricalEBMFactor([Block(nodes)], jnp.asarray(weights))


def make_interference_factor(
    graph: TVGraph, nodes: list[CategoricalNode], penalty: float
) -> CategoricalEBMFactor | None:
    seen: set[tuple[int, int, int, int]] = set()
    constraint_rows: list[tuple[int, int, int, int]] = []
    for station in graph.stations_by_id.values():
        if station.station_index is None:
            continue
        a_idx = station.station_index
        for interference, partner_idx in station.interferences_deduped():
            a_chan_idx = interference.subject_channel_index
            b_chan_idx = interference.other_channel_index
            key = (a_idx, partner_idx, a_chan_idx, b_chan_idx)
            if key in seen:
                raise ValueError(f"Duplicate constraint: {key}")
            seen.add(key)
            constraint_rows.append(key)

    if not constraint_rows:
        return None

    channel_count = graph.channel_count
    edge_count = len(constraint_rows)
    weights = np.zeros((edge_count, channel_count, channel_count), dtype=np.float32)
    head_nodes: list[CategoricalNode] = []
    tail_nodes: list[CategoricalNode] = []

    for idx, (a_idx, b_idx, a_chan_idx, b_chan_idx) in enumerate(constraint_rows):
        head_nodes.append(nodes[a_idx])
        tail_nodes.append(nodes[b_idx])
        weights[idx, a_chan_idx, b_chan_idx] = -float(penalty)

    return CategoricalEBMFactor([Block(head_nodes), Block(tail_nodes)], jnp.asarray(weights))


def prepare_initial_state(
    graph: TVGraph, free_blocks: list[Block], seed: int
) -> tuple[list[jnp.ndarray], int, int]:
    """Build initial state for blocks (each block may contain multiple nodes)."""
    key = jax.random.PRNGKey(seed)
    init_state: list[jnp.ndarray] = []
    post_count = 0
    random_count = 0
    
    for block in free_blocks:
        block_channels = []
        for node in block.nodes:
            # Find station index for this node
            station_index = None
            for idx, n in enumerate(nodes):
                if n is node:
                    station_index = idx
                    break
            
            if station_index is None:
                raise ValueError("Could not find station index for node")
            
            station_id = graph.station_id_for_index(station_index)
            station = graph.station(station_id)
            new_channel = station.new_channel
            
            if new_channel is not None:
                if new_channel not in graph.channel_values:
                    channel_idx = graph.channel_count - 1
                else:
                    channel_idx = graph.channel_index_for_channel(new_channel)
                post_count += 1
            else:
                domain = np.asarray(station.domain_indices, dtype=np.int32)
                if domain.size == 0:
                    raise ValueError(f"Station {station.station_id} has an empty domain.")
                key, subkey = jax.random.split(key)
                channel_idx = int(jax.random.choice(subkey, domain))
                random_count += 1
            
            block_channels.append(channel_idx)
        
        init_state.append(jnp.asarray(block_channels, dtype=jnp.uint8))
    
    return init_state, post_count, random_count


def block_assignment_to_array(block_state: list[jnp.ndarray]) -> np.ndarray:
    """Flatten block state to array (handles multi-node blocks)."""
    assignments = []
    for block in block_state:
        block_arr = np.asarray(block, dtype=np.int32)
        if block_arr.ndim == 0:
            assignments.append(int(block_arr))
        else:
            assignments.extend(block_arr.tolist())
    return np.asarray(assignments, dtype=np.int32)


def evaluate_energy(program: FactorSamplingProgram, factors: list[CategoricalEBMFactor], block_state: list[jnp.ndarray]) -> float:
    global_state = block_state_to_global(list(block_state), program.gibbs_spec)
    total = 0.0
    for factor in factors:
        total += float(factor.energy(global_state, program.gibbs_spec))
    return total


def score_assignment(graph: TVGraph, assignment: np.ndarray) -> dict[str, object]:
    channel_values = graph.channel_for_channel_id
    domain_mask = graph.domain_mask
    domain_violations = 0
    for station_index, channel_index in enumerate(assignment):
        if not domain_mask[station_index, channel_index]:
            domain_violations += 1

    violations = 0
    violations_by_type: Counter[str] = Counter()
    seen_pairs: set[tuple[int, int, int, int]] = set()

    for station in graph.stations_by_id.values():
        if station.station_index is None:
            continue
        a_idx = station.station_index
        assigned_a_idx = int(assignment[a_idx])
        assigned_a_val = channel_values[assigned_a_idx]

        for interference in station.interferences:
            if assigned_a_val != interference.subject_channel:
                continue
            for partner_idx in interference.station_indices:
                assigned_b_idx = int(assignment[partner_idx])
                assigned_b_val = channel_values[assigned_b_idx]
                if assigned_b_val != interference.other_channel:
                    continue

                if a_idx <= partner_idx:
                    key = (a_idx, partner_idx, assigned_a_idx, assigned_b_idx)
                    constraint_type = interference.constraint_type
                else:
                    key = (partner_idx, a_idx, assigned_b_idx, assigned_a_idx)
                    constraint_type = interference.constraint_type

                if key in seen_pairs:
                    continue
                seen_pairs.add(key)
                violations += 1
                violations_by_type[constraint_type] += 1

    return {
        "domain_violations": domain_violations,
        "violations": violations,
        "violations_by_type": violations_by_type,
    }

## Blocking Strategy Functions

In [ ]:
def create_single_node_blocks(nodes: list[CategoricalNode]) -> list[Block]:
    """Strategy 1: One node per block (baseline)."""
    return [Block([node]) for node in nodes]


def create_geographic_blocks(nodes: list[CategoricalNode], block_size: int) -> list[Block]:
    """Strategy 2: Group consecutive stations (assumes stations are ordered geographically)."""
    blocks = []
    for i in range(0, len(nodes), block_size):
        block_nodes = nodes[i:i+block_size]
        if block_nodes:  # Handle remainder
            blocks.append(Block(block_nodes))
    return blocks


def create_random_blocks(nodes: list[CategoricalNode], block_size: int, seed: int) -> list[Block]:
    """Strategy 3: Randomly shuffle and group stations."""
    shuffled = nodes.copy()
    random.seed(seed)
    random.shuffle(shuffled)
    
    blocks = []
    for i in range(0, len(shuffled), block_size):
        block_nodes = shuffled[i:i+block_size]
        if block_nodes:
            blocks.append(Block(block_nodes))
    return blocks

## Sampling Function

In [ ]:
def run_sampling(
    graph: TVGraph,
    free_blocks: list[Block],
    strategy_name: str,
    lambda_conflict: float,
    lambda_domain: float,
    warmup_steps: int,
    num_samples: int,
    seed: int,
) -> tuple[np.ndarray, dict, float, float]:
    """Run sampling with given blocking strategy and return best sample."""
    
    print(f"\n{'='*60}")
    print(f"Strategy: {strategy_name}")
    print(f"Number of blocks: {len(free_blocks)}")
    block_sizes = [len(block.nodes) for block in free_blocks]
    print(f"Block sizes: min={min(block_sizes)}, max={max(block_sizes)}, avg={np.mean(block_sizes):.1f}")
    print(f"{'='*60}")
    
    # Build factors
    domain_factor = make_domain_factor(graph, nodes, penalty=lambda_domain)
    conflict_factor = make_interference_factor(graph, nodes, penalty=lambda_conflict)
    factors = [domain_factor, conflict_factor] if conflict_factor else [domain_factor]
    
    # Build sampling program
    gibbs_spec = BlockGibbsSpec(
        free_super_blocks=free_blocks,
        clamped_blocks=[],
        node_shape_dtypes=DEFAULT_NODE_SHAPE_DTYPES,
    )
    samplers = [CategoricalGibbsConditional(graph.channel_count) for _ in free_blocks]
    program = FactorSamplingProgram(gibbs_spec, samplers, factors, other_interaction_groups=[])
    
    # Initialize
    init_state, post_count, random_count = prepare_initial_state(graph, free_blocks, seed)
    print(f"Initial state: {post_count} post-auction, {random_count} random")
    
    initial_assignment = block_assignment_to_array(init_state)
    init_energy = evaluate_energy(program, factors, init_state)
    init_stats = score_assignment(graph, initial_assignment)
    print(f"Initial: energy={init_energy:.2f}, violations={init_stats['violations']}, "
          f"domain breaches={init_stats['domain_violations']}")
    
    # Sample
    key = jax.random.PRNGKey(seed ^ 0xFEED5EED)
    state_free = [jnp.asarray(block) for block in init_state]
    clamp_state = []
    sampler_states = [sampler.init() for sampler in samplers]
    
    samples = []
    
    # Warmup
    for i in range(warmup_steps):
        key, subkey = jax.random.split(key)
        state_free, sampler_states = sample_blocks(subkey, state_free, clamp_state, program, sampler_states)
        if (i + 1) % 50 == 0:
            print(f"  Warmup step {i + 1}/{warmup_steps}")
    
    samples.append(block_assignment_to_array(state_free))
    
    # Sampling
    for i in range(1, num_samples):
        key, subkey = jax.random.split(key)
        state_free, sampler_states = sample_blocks(subkey, state_free, clamp_state, program, sampler_states)
        samples.append(block_assignment_to_array(state_free))
        if (i + 1) % 50 == 0:
            print(f"  Sample {i + 1}/{num_samples}")
    
    samples_array = np.stack(samples, axis=0)
    
    # Find best
    best_index = 0
    best_assignment = samples_array[0]
    best_stats = score_assignment(graph, best_assignment)
    best_energy = evaluate_energy(program, factors, [jnp.asarray([val], dtype=jnp.uint8) for val in best_assignment])
    
    for idx in range(1, len(samples_array)):
        assignment = samples_array[idx]
        stats = score_assignment(graph, assignment)
        energy = evaluate_energy(program, factors, [jnp.asarray([val], dtype=jnp.uint8) for val in assignment])
        if stats["violations"] < best_stats["violations"] or (
            stats["violations"] == best_stats["violations"] and energy < best_energy
        ):
            best_index = idx
            best_stats = stats
            best_energy = energy
    
    print(f"\nBest sample #{best_index}: energy={best_energy:.2f}, "
          f"violations={best_stats['violations']}, "
          f"domain breaches={best_stats['domain_violations']}")
    
    return best_assignment, best_stats, best_energy, init_energy

## Create Nodes and Run All Strategies

In [ ]:
# Create nodes (shared across all strategies)
nodes = [CategoricalNode() for _ in graph.station_ids_by_index]
print(f"Created {len(nodes)} categorical nodes")

### Strategy 1: Single-Node Blocks (Baseline)

In [ ]:
blocks_single = create_single_node_blocks(nodes)

best_single, stats_single, energy_single, init_energy_single = run_sampling(
    graph=graph,
    free_blocks=blocks_single,
    strategy_name="Single-Node Blocks",
    lambda_conflict=LAMBDA_CONFLICT,
    lambda_domain=LAMBDA_DOMAIN,
    warmup_steps=WARMUP_STEPS,
    num_samples=NUM_SAMPLES,
    seed=SEED,
)

### Strategy 2: Geographic Blocks

In [ ]:
blocks_geographic = create_geographic_blocks(nodes, BLOCK_SIZE_GEOGRAPHIC)

best_geo, stats_geo, energy_geo, init_energy_geo = run_sampling(
    graph=graph,
    free_blocks=blocks_geographic,
    strategy_name=f"Geographic Blocks (size={BLOCK_SIZE_GEOGRAPHIC})",
    lambda_conflict=LAMBDA_CONFLICT,
    lambda_domain=LAMBDA_DOMAIN,
    warmup_steps=WARMUP_STEPS,
    num_samples=NUM_SAMPLES,
    seed=SEED,
)

### Strategy 3: Random Blocks

In [ ]:
blocks_random = create_random_blocks(nodes, BLOCK_SIZE_RANDOM, SEED)

best_random, stats_random, energy_random, init_energy_random = run_sampling(
    graph=graph,
    free_blocks=blocks_random,
    strategy_name=f"Random Blocks (size={BLOCK_SIZE_RANDOM})",
    lambda_conflict=LAMBDA_CONFLICT,
    lambda_domain=LAMBDA_DOMAIN,
    warmup_steps=WARMUP_STEPS,
    num_samples=NUM_SAMPLES,
    seed=SEED,
)

In [ ]:
blocks_interference = create_interference_blocks(graph, nodes, BLOCK_SIZE_INTERFERENCE)

best_interference, stats_interference, energy_interference, init_energy_interference = run_sampling(
    graph=graph,
    free_blocks=blocks_interference,
    strategy_name=f"Interference Blocks (max_size={BLOCK_SIZE_INTERFERENCE})",
    lambda_conflict=LAMBDA_CONFLICT,
    lambda_domain=LAMBDA_DOMAIN,
    warmup_steps=WARMUP_STEPS,
    num_samples=NUM_SAMPLES,
    seed=SEED,
)

### Strategy 4: Interference-Based Blocks

## Compare Results

In [ ]:
import pandas as pd

results = pd.DataFrame([
    {
        "Strategy": "Single-Node Blocks",
        "Block Count": len(blocks_single),
        "Avg Block Size": 1.0,
        "Initial Energy": init_energy_single,
        "Best Energy": energy_single,
        "Energy Improvement": init_energy_single - energy_single,
        "Violations": stats_single["violations"],
        "Domain Breaches": stats_single["domain_violations"],
    },
    {
        "Strategy": f"Geographic Blocks (size={BLOCK_SIZE_GEOGRAPHIC})",
        "Block Count": len(blocks_geographic),
        "Avg Block Size": np.mean([len(b.nodes) for b in blocks_geographic]),
        "Initial Energy": init_energy_geo,
        "Best Energy": energy_geo,
        "Energy Improvement": init_energy_geo - energy_geo,
        "Violations": stats_geo["violations"],
        "Domain Breaches": stats_geo["domain_violations"],
    },
    {
        "Strategy": f"Random Blocks (size={BLOCK_SIZE_RANDOM})",
        "Block Count": len(blocks_random),
        "Avg Block Size": np.mean([len(b.nodes) for b in blocks_random]),
        "Initial Energy": init_energy_random,
        "Best Energy": energy_random,
        "Energy Improvement": init_energy_random - energy_random,
        "Violations": stats_random["violations"],
        "Domain Breaches": stats_random["domain_violations"],
    },
])

print("\n" + "="*80)
print("COMPARISON OF BLOCKING STRATEGIES")
print("="*80)
print(results.to_string(index=False))
print("="*80)

## Visualization Comparison

In [ ]:
%matplotlib inline

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

strategies = [
    ("Single-Node", stats_single),
    (f"Geographic (size={BLOCK_SIZE_GEOGRAPHIC})", stats_geo),
    (f"Random (size={BLOCK_SIZE_RANDOM})", stats_random),
]

for ax, (name, stats) in zip(axes, strategies):
    violations = stats["violations"]
    domain_breaches = stats["domain_violations"]
    
    categories = ['Violations', 'Domain\nBreaches']
    values = [violations, domain_breaches]
    colors = ['#e74c3c' if v > 0 else '#27ae60' for v in values]
    
    ax.bar(categories, values, color=colors, alpha=0.7, edgecolor='black')
    ax.set_title(f"{name}\n({violations} violations)", fontsize=12, fontweight='bold')
    ax.set_ylabel('Count')
    ax.grid(axis='y', alpha=0.3)
    
    # Add value labels on bars
    for i, (cat, val) in enumerate(zip(categories, values)):
        ax.text(i, val + max(values) * 0.02, str(val), ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n📊 Interpretation:")
print("- Lower violations = better solution quality")
print("- Multi-node blocks may converge faster but with different final quality")
print("- Geographic blocks leverage spatial structure")
print("- Random blocks provide mixing without spatial bias")